## Full Pipeline: BETO Multi-Class Classification on Translated Spanish Tweets

In [ ]:
!pip install -U transformers datasets scikit-learn optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 124.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 129.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4


**STEP 1:** Load & Filter Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Load dataset as a DataFrame
df = pd.read_csv('/content/drive/MyDrive/traducido_dataset_unificado_v_30jun.csv')

# Preview the dataset
df

/tmp/ipython-input-3-3159178268.py:4: DtypeWarning: Columns (9,10,11,12,13,19,20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/traducido_dataset_unificado_v_30jun.csv')


,source_dataset,target_column,subcategory_combined,text,text_english,text_spanish,id,subcategory_general,subcategory_specific,rewire_id,...,TR,AG,label,test_case,source,language,task1,task2,detected_language,multiclass_final
0,edos_labelled_aggregated,0,NaN,"In Nigeria, if you rape a woman, the men rape ...","In Nigeria, if you rape a woman, the men rape ...","En Nigeria, si violas a una mujer, ¡los hombre...",NaN,NaN,NaN,sexism2022_english-9609,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,NaN
1,edos_labelled_aggregated,0,NaN,"Then, she's a keeper. 😉","Then, she's a keeper. 😉","Entonces, ella es una guardiana. 😉",NaN,NaN,NaN,sexism2022_english-16993,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,NaN
2,edos_labelled_aggregated,0,NaN,This is like the Metallica video where the poo...,This is like the Metallica video where the poo...,Esto es como el vídeo de Metallica donde el po...,NaN,NaN,NaN,sexism2022_english-13149,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,NaN
3,edos_labelled_aggregated,0,NaN,woman?,woman?,¿mujer?,NaN,NaN,NaN,sexism2022_english-13021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,so,NaN
4,edos_labelled_aggregated,0,NaN,I bet she wished she had a gun,I bet she wished she had a gun,Apuesto a que deseaba tener un arma.,NaN,NaN,NaN,sexism2022_english-966,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44347,EXIST2021_training,0,NaN,"Estamos igual sin pareja, pero puedes besar a ...","We're the same without a partner, but you can ...","Estamos igual sin pareja, pero puedes besar a ...",6973.0,NaN,NaN,NaN,...,NaN,NaN,NaN,EXIST2021,twitter,es,non-sexist,non-sexist,es,NaN
44348,EXIST2021_training,0,NaN,2020 hijo de re mil putas,2020 son of a bitch,2020 hijo de re mil putas,6974.0,NaN,NaN,NaN,...,NaN,NaN,NaN,EXIST2021,twitter,es,non-sexist,non-sexist,es,NaN
44349,EXIST2021_training,0,NaN,SEGURAMENTE ESTA CHICA NO COBRA EL DINERO QUE ...,SURELY THIS GIRL DOESN'T GET PAID THE MONEY SH...,SEGURAMENTE ESTA CHICA NO COBRA EL DINERO QUE ...,6975.0,NaN,NaN,NaN,...,NaN,NaN,NaN,EXIST2021,twitter,es,non-sexist,non-sexist,pt,NaN
44350,EXIST2021_training,1,NaN,@safetyaitana mi madre dice q va fea y i agree,@safetyaitana my mother says she looks ugly an...,@safetyaitana mi madre dice q va fea y estoy d...,6976.0,NaN,NaN,NaN,...,NaN,NaN,NaN,EXIST2021,twitter,es,sexist,objectification,ca,NaN


In [ ]:
print(df["multiclass_final"].value_counts())

multiclass_final
3. General Hostile Language            6651
2. Gendered Stereotypes & Insults       964
1. Sexual Insults & Objectification     447
4. Threats & Physical Harm              443
5. Victim Blaming & Justification       107
Name: count, dtype: int64


In [ ]:
df_es = df[df['detected_language'] == 'es']
df_es

,source_dataset,target_column,subcategory_combined,text,text_english,text_spanish,id,subcategory_general,subcategory_specific,rewire_id,...,TR,AG,label,test_case,source,language,task1,task2,detected_language,multiclass_final
1821,edos_labelled_aggregated,0,NaN,I am gonna go inside your mom,I am gonna go inside your mom,Voy a entrar dentro de tu mamá,NaN,NaN,NaN,sexism2022_english-5866,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,es,NaN
6036,edos_labelled_aggregated,0,NaN,tfw no spectral bride,tfw no spectral bride,tfw sin novia espectral,NaN,NaN,NaN,sexism2022_english-16831,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,es,NaN
13006,edos_labelled_aggregated,0,NaN,steroids lol you retarded,steroids lol you retarded,"Esteroides jajaja, retrasado mental",NaN,NaN,NaN,sexism2022_english-5192,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,es,NaN
16751,edos_labelled_aggregated,0,NaN,"Welcome to #Gab, ladies!","Welcome to #Gab, ladies!","¡Bienvenidas a #Gab, damas!",NaN,NaN,NaN,sexism2022_english-10570,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,es,NaN
16897,edos_labelled_aggregated,0,NaN,lmao go fuck yourself cuck,lmao go fuck yourself cuck,"jajaja vete a la mierda, cornudo",NaN,NaN,NaN,sexism2022_english-13972,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,es,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44345,EXIST2021_training,0,NaN,@SamuelGuidoIII @FuenzaH El análisis de causa ...,@SamuelGuidoIII @FuenzaH The cause analysis is...,@SamuelGuidoIII @FuenzaH El análisis de causa ...,6971.0,NaN,NaN,NaN,...,NaN,NaN,NaN,EXIST2021,twitter,es,non-sexist,non-sexist,es,NaN
44346,EXIST2021_training,0,NaN,@Jony_4k @kkkorel yo también colaboro con la g...,@Jony_4k @kkkorel I also collaborate with peop...,@Jony_4k @kkkorel yo también colaboro con la g...,6972.0,NaN,NaN,NaN,...,NaN,NaN,NaN,EXIST2021,twitter,es,non-sexist,non-sexist,es,NaN
44347,EXIST2021_training,0,NaN,"Estamos igual sin pareja, pero puedes besar a ...","We're the same without a partner, but you can ...","Estamos igual sin pareja, pero puedes besar a ...",6973.0,NaN,NaN,NaN,...,NaN,NaN,NaN,EXIST2021,twitter,es,non-sexist,non-sexist,es,NaN
44348,EXIST2021_training,0,NaN,2020 hijo de re mil putas,2020 son of a bitch,2020 hijo de re mil putas,6974.0,NaN,NaN,NaN,...,NaN,NaN,NaN,EXIST2021,twitter,es,non-sexist,non-sexist,es,NaN


In [ ]:
# Filter for:
# 1. Labeled as sexist (target_column == 1)
# 2. Have text in Spanish
# 3. Have a defined multiclass label

df_filtered = df[
    (df["target_column"] == 1) &  # sexist tweets
    (df["text_spanish"].notnull()) &  # usable Spanish text
    (df["multiclass_final"].notnull())  # valid label
].copy()

# Sample only 20% for grid search
df_filtered = df_filtered.sample(frac=0.2, random_state=42).reset_index(drop=True)

**STEP 2:** Encode Labels

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode multiclass_final into numeric labels
le = LabelEncoder()
df_filtered["label"] = le.fit_transform(df_filtered["multiclass_final"])

# Save mapping for later use
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(label_mapping)

{'1. Sexual Insults & Objectification': np.int64(0), '2. Gendered Stereotypes & Insults': np.int64(1), '3. General Hostile Language': np.int64(2), '4. Threats & Physical Harm': np.int64(3), '5. Victim Blaming & Justification': np.int64(4)}


In [ ]:
# Create a vertical view of the label mapping
import pandas as pd

pd.DataFrame(label_mapping.items(), columns=["multiclass_final", "label"])

,multiclass_final,label
0,1. Sexual Insults & Objectification,0
1,2. Gendered Stereotypes & Insults,1
2,3. General Hostile Language,2
3,4. Threats & Physical Harm,3
4,5. Victim Blaming & Justification,4


In [ ]:
df_filtered["multiclass_final"].value_counts(normalize=True)

,proportion
multiclass_final,
3. General Hostile Language,0.781649
2. Gendered Stereotypes & Insults,0.113821
1. Sexual Insults & Objectification,0.047038
4. Threats & Physical Harm,0.044715
5. Victim Blaming & Justification,0.012776


In [ ]:
print(df_filtered["multiclass_final"].value_counts())

multiclass_final
3. General Hostile Language            1346
2. Gendered Stereotypes & Insults       196
1. Sexual Insults & Objectification      81
4. Threats & Physical Harm               77
5. Victim Blaming & Justification        22
Name: count, dtype: int64


**STEP 3:** Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df_filtered["text_spanish"],
    df_filtered["label"],
    test_size=0.2,
    stratify=df_filtered["label"],
    random_state=42
)

**STEP 4:** Tokenization with BETO

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer

# Convert to HuggingFace Datasets using the original column name
train_data = Dataset.from_pandas(pd.DataFrame({"text_spanish": X_train, "label": y_train}))
test_data = Dataset.from_pandas(pd.DataFrame({"text_spanish": X_test, "label": y_test}))

# Load BETO tokenizer
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

# Use correct column for tokenization
def tokenize_function(examples):
    return tokenizer(examples["text_spanish"], padding="max_length", truncation=True, max_length=512)

# Tokenization function
train_dataset = train_data.map(tokenize_function, batched=True)
test_dataset = test_data.map(tokenize_function, batched=True)

# Format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

Map:   0%|          | 0/1377 [00:00<?, ? examples/s]

Map:   0%|          | 0/345 [00:00<?, ? examples/s]

**STEP 5:** Check Class Distribution in Train Set

In [ ]:
import pandas as pd

# See how balanced the training data is
pd.Series(y_train).value_counts(normalize=True)

,proportion
label,
2,0.781409
1,0.114016
0,0.047204
3,0.044299
4,0.013072


**STEP 6:** Define Evaluation Metrics

In [ ]:
from transformers import EvalPrediction
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(p: EvalPrediction):
    preds = p.predictions.argmax(axis=1)
    return {
        'accuracy': accuracy_score(p.label_ids, preds),
        'precision_macro': precision_score(p.label_ids, preds, average='macro', zero_division=0),
        'recall_macro': recall_score(p.label_ids, preds, average='macro', zero_division=0),
        'f1_macro': f1_score(p.label_ids, preds, average='macro'),
        'f1_weighted': f1_score(p.label_ids, preds, average='weighted')
    }

**STEP 7:** Define Model Initialization and Hyperparameter Search Space

In [ ]:
from transformers import BertForSequenceClassification
import optuna

def model_init():
    return BertForSequenceClassification.from_pretrained(
        "dccuchile/bert-base-spanish-wwm-cased",
        num_labels=len(label_mapping)
    )

def hp_space(trial):
    return {
        "learning_rate": trial.suggest_categorical("learning_rate", [2e-5, 3e-5, 5e-5]),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 2, 4),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16]),
        "weight_decay": trial.suggest_float("weight_decay", 0.0, 0.3),
    }

**STEP 8:** Run Grid Search with Optuna

In [ ]:
import json
from transformers import Trainer, TrainingArguments

# Step 8.1: Define TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    report_to="none",  # or "wandb" if you're using Weights & Biases
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    disable_tqdm=False
)

# Step 8.2: Initialize Trainer
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Step 8.3: Run Optuna hyperparameter search
best_run = trainer.hyperparameter_search(
    direction="maximize",
    hp_space=hp_space,
    n_trials=4,
    backend="optuna"
)

# Step 8.4: Save results to Google Drive
# ⚠️ Make sure your drive is mounted using: drive.mount('/content/drive') earlier

best_path = "/content/drive/MyDrive/BETO/beto_gridsearch_best_results_multiclass.json"
trials_path = "/content/drive/MyDrive/BETO/beto_gridsearch_all_trials_multiclass.txt"

# Save the best result
with open(best_path, "w") as f:
    json.dump({
        "best_score": best_run.objective,
        "best_hyperparameters": best_run.hyperparameters
    }, f, indent=4)

# Save all trial logs
with open(trials_path, "w") as f:
    for trial in best_run.trials:
        f.write(f"Trial {trial.number} | F1_macro: {trial.value:.4f} | Params: {trial.params}\n")

print("✅ Best run and all trials saved to Google Drive.")

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[I 2025-06-30 19:01:23,212] A new study created in memory with name: no-name-4c0ba5dc-908f-4b6f-bfdb-516804528ba7


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision Macro,Recall Macro,F1 Macro,F1 Weighted
1,No log,0.561217,0.797101,0.315079,0.234416,0.235231,0.725179
2,No log,0.483753,0.823188,0.444568,0.377628,0.401144,0.785939
3,No log,0.484097,0.814493,0.443935,0.348903,0.375638,0.779678


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision Macro,Recall Macro,F1 Macro,F1 Weighted
1,No log,0.553734,0.820290,0.517650,0.324060,0.365209,0.769971
2,0.640700,0.536292,0.814493,0.473035,0.432621,0.443347,0.807883
3,0.321000,0.687559,0.828986,0.538523,0.405434,0.444600,0.808907


[I 2025-06-30 19:28:25,869] Trial 2 finished with value: 3.0264507987186686 and parameters: {'learning_rate': 2e-05, 'num_train_epochs': 3, 'per_device_train_batch_size': 4, 'weight_decay': 0.12015371681065917}. Best is trial 1 with value: 3.2479809838171754.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision Macro,Recall Macro,F1 Macro,F1 Weighted
1,No log,0.613946,0.797101,0.558824,0.240385,0.250065,0.719256
2,No log,0.574760,0.794203,0.532762,0.398127,0.386437,0.781336
3,No log,0.673172,0.823188,0.764777,0.464972,0.531000,0.811372
4,No log,0.748143,0.811594,0.600015,0.566496,0.573046,0.801229


[I 2025-06-30 19:39:44,746] Trial 3 finished with value: 3.3523796419503316 and parameters: {'learning_rate': 5e-05, 'num_train_epochs': 4, 'per_device_train_batch_size': 16, 'weight_decay': 0.11569788355176508}. Best is trial 3 with value: 3.3523796419503316.


AttributeError: 'BestRun' object has no attribute 'trials'

In [ ]:
import json

# Save best trial only
best_path = "/content/drive/MyDrive/BETO/beto_gridsearch_best_results.json"

with open(best_path, "w") as f:
    json.dump({
        "best_score": best_run.objective,
        "best_hyperparameters": best_run.hyperparameters
    }, f, indent=4)

print("✅ Best run saved to Google Drive.")

✅ Best run saved to Google Drive.


In [ ]:
print("🔍 Best Trial Hyperparameters:")
for param, value in best_run.hyperparameters.items():
    print(f"{param}: {value}")

print(f"\n🏆 Best F1_macro Score: {best_run.objective:.4f}")

🔍 Best Trial Hyperparameters:
learning_rate: 5e-05
num_train_epochs: 4
per_device_train_batch_size: 16
weight_decay: 0.11569788355176508

🏆 Best F1_macro Score: 3.3524
